# GDA – Übung 4: Visualsierung

## Einleitung
In dem Hauptteil der heutigen Übung betrachten wir die Methoden *Kernel Density Estimation (KDE)* und *Inverse Distance Weighting (IDW)*. Während Erstere ein Verfahren zur Schätzung der Wahrscheinlichkeitsverteilung einer Zufallsvariablen ist, ist Letzere eine in der GIS-Welt häufig angewandte Interpolationsmethode, um Werte zwischen bekannten Messwerten zu interpolieren. Es geht wie häufig in der Welt hauptsächlich um das erste geographische Gesetz nach Waldo Tobler:  
> Everything is related to everything else, but near things are more related than distant things.
>
> -- <cite>Tobler W., (1970) "A computer movie simulating urban growth in the Detroit region". Economic Geography, 46(Supplement): 234–240.</cite>

Angenommen, es existiert ein räumliches Phänomen, aber Sie sind (aus wirtschaftlichen oder sonstigen Gründen) nur imstande, einige Punkte zu messen und daher eine Stichprobe zu nehmen, so müssen Sie die dazwischenliegenden Werte interpolieren. Das Ziel der heutigen Übung ist es, beide Verfahren selbstständig zu implementieren. Dabei sollen Sie auch unterschiedliche Kernels anwenden und deren Auswirkungen auf die resultierenden Karten untersuchen. 

### Dateien
Wir verwenden für diese Aufgabe den *Meuse*-Datensatz (siehe http://spatial-analyst.net/book/meusegrids), der von Burrough und McDonnel 1998 erstellt wurde. Der Datensatz umfasst vier Schwermetalle im Oberboden (Einheit: **ppm**) und andere Kovariaten, die entlang des Flusses *Meuse* in der oberen Bodenschicht gemessen wurde. Es scheint so, als ob der Fluss die mit Schwermetall belasteten Sedimente flussabwärts transportieren und entlang des Flusses und im Flussdelta ablagern würde. Im Sinne von Tobler's First Law of Geography dürfte man deshalb erwarten, dass die Schwermetallwerte im ganzen Untersuchungsgebiet nicht gleich verteilt sind.
> -- <cite>P.A. Burrough, R.A. McDonnell, 1998. Principles of Geographical Information Systems. Oxford University Press.</cite>

In dieser Aufgabe erhalten Sie die folgenden Dateien:
* `meuse.json` ein JSON mit Messwerten mit Schwermetallen, die an verschiedenen Puntken entlang des Flusses Meuse aufgezeichnet wurden. Darin sind u.a. folgende Attribute enthalten:
  * cadmium: Cd-Gehalt in ppm
  * copper: Cu-Gehalt in ppm
  * lead: Pb-Gehalt in ppm
  * zinc: Zn-Gehalt in ppm
  * dist: Distanz zum Ufer der Meuse
  * ffreq: jährliche Häufigkeit von Überflutungen
  * soil: 
* `mask.tif` ein leeres Raster im Flussdelta des Untersuchungsgebiet in einer Auflösung von 25 m. Alle Pixel mit dem Wert 1 kennzeichnen Pixel innerhalb des Studiengebiets. Alle anderen Werte sind dem `nodata`-Wert zugeordnet und gehören nicht zum Untersuchungsgebiet.
* `topomap2m` eine topographische Karte des Untersuchungsgebiets in einer Auflösung von 2 m. Diese dient Ihnen zur Übersicht.

Die Dateien sind allesamt im folgenden projizierten CRS gespeichert: **proj4: +init=epsg:28992**


### Hilfsfunktionen

In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import fiona
import math
import random
from pprint import pprint
import pandas as pd
import geopandas as gpd
import json
from tqdm import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from rasterio.transform import Affine
from sklearn.neighbors import NearestNeighbors

In [ ]:
def show_raster_map_grid(array_list, mask=None, data_coordinates=None, dim=None):
    """
    Plots 1 to n arrays next to each other
    :param array_list: A list of arrays (images). Has to be a list, can be of length 1 to plot only a single image
    :mask (optional): Every array in `array_list` will be multiplied with mask before plotting. 
    :param data_coordinates: (optional) list of xy coordinate tuples to indicate the place of measurement (in true coords)
    :param dim: (optional) Tuple that indicates the ordering of the plots (e.g., dim=(2, 6) creates a 2x6 grid for 12 plots)
    :return:
    """
    
    # if no ordering provided, arrange all plots in a row
    if dim is None:
        dim = (1, len(array_list))
        
    fig, axes = plt.subplots(dim[0], dim[1], figsize=(20, 20))
    
    # If we only have 1 plot we have to tranform the `axes` into an iterable object
    try:
        axes[0]
    except TypeError:
        axes = (axes,)
    
    # create a plot for every ax in axes
    for ix, ax in enumerate(axes):
    
        array = array_list[ix]
        
        if mask is None:
            mask = np.ones(array.shape)
        
        im = ax.imshow(array*mask, cmap='hot')  
        
        # if provided, plot the query points.
        # Query points first have to be projected into the image space
        if data_coordinates is not None:
            rows = []
            cols = []
            for i in range(len(data_coordinates)):
                #https://rasterio.readthedocs.io/en/latest/api/rasterio.transform.html#rasterio.transform.rowcol
                rowcol_tuple = rasterio.transform.rowcol(fwd, data_coordinates[i][0], data_coordinates[i][1])
                rows.append(rowcol_tuple[0])
                cols.append(rowcol_tuple[1]) 


            ax.scatter(cols, rows, color='b', s=30, marker='x')
        
        # special trick for colorbar (https://stackoverflow.com/questions/32462881/add-colorbar-to-existing-axis)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')

    plt.show()
        
    
def compute_distance(coords_1, coords_2):
    """
    Computes the euclidean distance between two coordinates given that they are represented by projected coordinates
    :param coord_1: A tuple or list containing a coordinate pair
    :param coord_2: A tuple or list containing a coordinate pair
    :return: The distance given in the same length dimension as they are stored in the used file
    """
    delta_y = coords_1[0] - coords_2[0]
    delta_x = coords_1[1] - coords_2[1]
    return math.sqrt(math.pow(delta_x, 2) + math.pow(delta_y, 2))

def get_transform(xmax, xmin, ymax, ymin, nb_pixels_x, nb_pixels_y):

    # define transformation function
    resx = (xmax - xmin) / nb_pixels_x
    resy = (ymax - ymin) / nb_pixels_y
    fwd = Affine.translation(xmin - resx / 2, ymin - resy / 2) * Affine.scale(resx, resy)

    return fwd


### Daten laden und wichtige Variablen definieren

In [ ]:
# Define important paths
meuse_file = os.path.join('data', 'meuse.json')
mask_file = os.path.join('data', 'mask.tif')
topomap_file = os.path.join('data', 'topomap2m.tif')

# Load the data sets
meuse = gpd.read_file(meuse_file)
mask = rasterio.open(mask_file, 'r')
topomap = rasterio.open(topomap_file, 'r')

# Read the first band of the ones raster
mask_array = mask.read(1)
mask_array[mask_array <= 0] = np.nan


# Determine other important settings from the input raster
no_data = mask.profile['nodata']
fwd = mask.profile['transform']
crs = mask.profile['crs']

### Erkundung der Daten: Look at the data

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,10))

rgb_array = [topomap.read(i+1) for i in range(3)]

ax[0].imshow(np.stack(rgb_array, axis=-1), cmap='gray')
ax[0].set_title('Map of region')
ax[1].imshow(mask_array)
ax[1].set_title('Mask')
meuse.plot(column='zinc', ax=ax[2])
ax[2].set_title('Sample locations')

### Daten in einfachem Format vorbereiten
- __data_coordinates__ ist eine Liste mit (x,y) Koordinatentupeln.
- __zinc_data__ ist eine Liste mit den Zinkwerten die an den Punkten gemessen wurden.

Die einzelnen Einträge der beiden Variablen gehören jeweils zusammen. Beispiel:
data_coordinates[i] sind die Koordinaten an denen zinc_data[i] gemessen wurde.

In [ ]:
coords_x = meuse.geometry.x
coords_y = meuse.geometry.y
zinc_data =  meuse['zinc'].values

data_coordinates = list(zip(coords_x, coords_y))
data_coordinates

In [ ]:
zinc_data =  meuse['zinc'].values
zinc_data

# Aufgabe 1: KDE mit Gauss Kernel
Implementieren in der Funktion `kde` eine Kernel Density Estimation mit einem Gauss Kernel. Orientieren Sie sich dafür anhand der folgenden Formeln:

$$f_h(x) = \frac{1}{n\cdot h} \sum^n_{i=n}K(\frac{d(x,x_i)}{h})$$


$$K(u) = \frac{1}{\sqrt{2\pi}}\exp(-\frac{u^2}{2})$$

Verwenden sie die Zinkkonzentration als Gewichtung. Wo befinden sich Ihrer Meinung nach die höchsten Konzentrationen an Zink?




### Vorgehen:
Sehen Sie sich die Hauptfunktion weiter unten an, hier wird auf die Funktion `kde` mit den einzelnen Pixelkoordinaten als Eingangswert zugegriffen. Vervollständigen Sie die Funktion `kde`, sodass für jeden Eingangspixel der Einfluss aller Messwerte basierend auf der obigen Formel berechnet wird.

* Iterieren Sie für jeden Pixel durch alle verfügbaren Datenpunkte (`data_coordinates`)
* Berechnen Sie den Einfluss eines jeden Datenpunktes mit der KDE Formel oben und summieren Sie diesen Wert für alle verfügbaren Datenpunkte.
* Gewichten Sie den Einfluss eines jeden Datenpunktes mit der Höhe der Zink konzentration

#### Hinweise:
- mask_array beschreibt die Ausdehnung des Studiengebiets in Pixelkoordinaten
- Vergessen Sie nicht den Eingangspixel (=`cell_position`) in den Koordinatenraum zu projezieren um die Distanz zu berechnen. Mit der durch rasterio vorgegeben transformation (`fwd`) ist das bereits implementiert und vorbereitet  (https://rasterio.readthedocs.io/en/latest/quickstart.html#dataset-georeferencing).


In [ ]:
def kde(cell_position, data_coordinates=data_coordinates, data_values=zinc_data, h=100, pixel_to_coords_transform=fwd):
    """
    Computes the kernel density estimate at a single cell position. 
    :param pixel_coordinates: A tuple containing the xy coordinates of a single pixel in pixel coordinates.
    :param data_points: List of x-y coordinate tuples of the data_values
    :param data_values: List of data values used for weighting the results
    :param h: Bandwidth parameter
    :param fwd: The Affine object by which the forward transformation can be executed
    :return: The result that should be written into the corresponding pixel
    """
    # Convert the pixel coordinates to real coordinates by using the fwd transformation
    cell_coordinates = pixel_to_coords_transform * cell_position
    
    # your code here
    
    
    
    
    
    result = np.random.uniform() # replace this by the true value
    
    return result

# test. Der Wert sollte in etwa bei 3.247539587692823e-86 liegen
kde((0,2), data_coordinates, zinc_data)

### Hauptfunktion Aufgabe 1

In [ ]:
kde_array = np.zeros(mask_array.shape)

# Iterate all pixels. Pixels outside the mask can be ignored.
for y in tqdm(range(mask_array.shape[0])):
    for x in range(mask_array.shape[1]):
        
        if not np.isnan(mask_array[y][x]):
            kde_array[y][x] = kde((x, y))
            
        else:
            kde_array[y][x] = -1
            
show_raster_map_grid([kde_array], mask_array, data_coordinates=data_coordinates)

# Aufgabe 2: Inverse Distance Weighting
Implementieren sie IDW anhand der folgenden Formeln:




$$ u(x) =  \begin{cases} 
      \frac{\sum_{i=1}^n w_i(x)\cdot u_i}{\sum^N_{i=1} w_i(x)} & if d(x,x_i) = 0  \\
      u_i & if d(x,x_i) = 0 
   \end{cases}
$$

$$ w_i(x) = \frac{1}{d(x, x_i)^p}$$



Verwenden sie die Zinkkonzentration als Gewichtung. Wo befinden sich Ihrer Meinung nach die höchsten Konzentrationen an Zink?

In [ ]:
def idw(cell_position, data_coordinates=data_coordinates, data_values=zinc_data,  p=2, pixel_to_coords_transform=fwd):
    """
    Computes the value at a single pixel position using inverse distance weighting.
    :param pixel_coordinates: A tuple containing the xy coordinates of a single pixel in pixel coordinates.
    :param data_points: List of x-y coordinate tuples of the data_values
    :param data_values: List of data values used for weighting the results
    :param p: Distance decay paramter
    :param fwd: The Affine object by which the forward transformation can be executed
    :return: The result that should be written into the corresponding pixel
    """
    
    # Convert the pixel coordinates to real coordinates by using the fwd transformation
    cell_coordinates = pixel_to_coords_transform * cell_position
    
    # your code here
    
    
    
    
    
    result = np.random.uniform() # replace this by the true value
    
    return result


# Test, value should be close to 518.8726477986377
idw((0,2), data_coordinates, zinc_data)

### Hauptfunktion Aufgabe 2
Erstellen Sie ähnlich wie in Aufgabe 1 eine neue Matrix und Berechnen Sie die Werte aller Pixel mit IDW. Pixel ausserhalb der Maske können ignoriert werden.
- Verwenden Sie verschiedene Werte für p. Welchen Einfluss hat dieser Paramter und welche Annahmen trifft man mit der Wahl des Paramters?

In [ ]:
idw_array = np.zeros(mask_array.shape)

# Iterate through the ones array and consider just the values which are not no_data
for y in tqdm(range(mask_array.shape[0])):
    for x in range(mask_array.shape[1]):
        
        if mask_array[y][x] != 0:
            idw_array[y][x] = idw((x, y), p=4)
            
        
        else:
            idw_array[y][x] = -1
            
show_raster_map_grid([kde_array], mask_array, data_coordinates=data_coordinates)

### Aufgabe 3: Vergleich und Verständnis für Hyperparameter
Übergeben Sie der Funktion `show_raster_map_grid` eine Liste mit mehreren Matrizen aus dem Studiengebiet, um sie nebeneinander darzustellen und zu vergleichen.
- Was sind die Unterschiede der beiden Methoden?
- Welche würden Sie wofür verwenden? 
- Wenn Sie einen Ort mit einer möglichst geringen Zinkkonzentration suchen, welche Methode ist besser geeignet?

In [ ]:
show_raster_map_grid([kde_array, idw_array], mask_array, data_coordinates=data_coordinates)



# Aufgabe 4: Heatmaps für Trajektorien (Bonus)
In den letzten Übungen haben wir uns mit Trajektoriendaten beschäftigt. Das Zeichnen einer grossen Menge an Trajektoriendaten wird in der Praxis schnell unübersichtlich. Hier sind Beispieldaten aus dem Geolife Datensatz. Die ist ein GPS Datensatz, bei dem die Bewegungen von Mitarbeitern von Microsoft Research in Peking über mehrere Monate hinweg aufgezeichnet wurden. Der Datensatz ist [frei verfügbar](https://www.microsoft.com/en-us/download/details.aspx?id=52367).

Der folgende Abschnitt zeichnet alle Trajektorien leicht durchsichtig übereinander (Alpha Blending).
- Können Sie erkennen, welches Cluster am relevantesten ist?

In [ ]:
crs_wgs = 'epsg:4326'
crs_2D = 'epsg:32650'
tpls_all = gpd.read_file(os.path.join('data','shp', 'geolife_triplegs.shp'))
tpls_all = tpls_all.to_crs(crs_2D)

xmin = 420000
xmax = 466000
ymin = 4450000
ymax = 4400000

tpls_all.plot(figsize=(12,12), alpha=0.3)
plt.xlim((xmin, xmax))
plt.ylim((ymin, ymax))

### 4.1 Tranformation der Trajektoriendaten
KDE kann auch für die Visualisierung von Trajektoriendaten verwendet werden. Eine Möglichkeit ist dabei, die Trajektorie in regelmässigen Abständen abzutasten und auf den Abtastpunkten das gleiche KDE-Verfahren wie in Aufgabe 1 anzuwenden. Für diese Übung können Sie annehmen, dass die Trajektorien bereits regelmässig gesampled wurden. Sie können also direkt die Trajektorien aus dem  Geopandas Dataframe in eine Liste mit Koordinatentupeln transformieren und Ihre oben implementierten Funktion übergeben.

__Achtung__: Bei Trajektorien sammeln sich schnell grosse Datenmengen an! Nutzen Sie `tpls_all.sample(10)`, um ihren Code  mit wenigen Daten effizient zu testen.
__Achtung II__: Wenn Sie den Code später mit mehr Trajektorien ausprobieren wollen, müssen Sie hier mehr Trajektorien samplen.


In [ ]:
tpls = tpls_all.sample(5)

# your code here
    
    


### 4.2 KDE Berechnung
Wenn die Punkte aus den Trajektorien extrahiert wurden, können wir das KDE ähnlich wie in der Hauptfunktion von Aufgabe 1 berechnen.

#### Hinweise: 
- Sie können die Auflösung (über die Anzahl an Pixeln in x und y Richtung selbst definieren. Benutzen Sie zu Beginn maximal eine Auflösung von 30x30. Wenn Ihr Code funktioniert, können Sie die Auflösung erhöhen. Eine Erhöhung übersteigt schnell die Rechenzeit!
- Für den neuen Datensatz brauchen wir später eine neue Pixel-zu-Koordinaten Transformationsfunktion. Diese haben wir Ihnen bereits vorgegeben.
- Nutzen Sie unbedingt tqdm(range(nb_pixels_y)) in der äussersten Schleife um ein Gefühl für die Rechenzeit des Algorithmus zu bekommen.


In [ ]:
# create transformation

nb_pixels_x = 30
nb_pixels_y = 30


fwd_geolife = get_transform(xmax, xmin, ymax, ymin, nb_pixels_x, nb_pixels_y)

# init
kde_array_traj = np.zeros((nb_pixels_x ,nb_pixels_y)) 

# your code here
        

###  Aufgabe 5: Effiziente Kernel Density Estimation mithilfe des Nearest Neighbours Algorithmus
Sie werden bei der Aufgabe festgestellt haben, dass die Rechenzeit bereits für nur 10 Trajektorien und eine sehr grobe räumliche Auflösung hoch ist. Das liegt vor allem an der ineffizienten Berechnung. In unserer `kde` Funktion wird für jeden Pixel jeder verfügbare Datenpunkt angesehen und fliesst mit in die Berechnung ein, egal wie weit er weg ist. Diese Genauigkeit und Gründlichkeit hat in der Praxis kaum Vorteile, da die nächsten Nachbarn in der Regel den grössten Einfluss haben. Mit einer sehr einfachen Nearest Neighbours basierten Anpassung lässt sich der Rechenaufwand schnell um mehrere Grössenordnungen reduzieren. 

Hinweise:
- Verwenden Sie Klasse [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors) aus dem scikit-learn Modul. Verwenden Sie ein Objekt dieser Klasse um einen Index für die `data_coordinates` zu erstellen.
- Im Anschluss können Sie sich über `distances, indices = nbrs.kneighbors(cell_coordinates)` die Distanzen zu den `n` nächsten Nachbarn ausgeben lassen. Im Anschluss können Sie über den Array `distances` iterieren anstatt die Distanz zu allen Datenpunkten auszurechnen.


In [ ]:
def kde_nn(cell_position, data_coordinates, nn, h=100, pixel_to_coords_transform=fwd, weight=1):
    """
    Computes the kernel density estimate at a single cell position. 
    :param pixel_coordinates: A tuple containing the xy coordinates of a single pixel in pixel coordinates.
    :param data_points: List of x-y coordinate tuples of the data_values
    :param data_values: List of data values used for weighting the results
    :param h: Bandwidth parameter
    :param fwd: The Affine object by which the forward transformation can be executed
    :return: The result that should be written into the corresponding pixel
    """
    # Convert the pixel coordinates to real coordinates by using the fwd transformation
    cell_coordinates = pixel_to_coords_transform * cell_position
    
    # your code here
    
    
    
    
    
    result = np.random.uniform() # replace this by the true value
    
    return result



Kopieren Sie hier die Hauptfunktion aus Aufgabe 4 und passen Sie sie so an, dass das NN Objekt erstellt wird und Sie im Anschluss die KDE Werte mit der neuen NN basierten Funktion berechnen können. Für die Visualisierung ist es hilfreich eine logarithmische Skala zu verwenden (z.B. mit `np.log(kde_array_traj_nn+1)`).
- Wenn der Test mit wenigen Trajektorien erfolgreich war, können Sie diese Funktion ohne Probleme auf alle Trajektorien Anwenden (Anzahl Pixel langsam steigern).
- Können Sie die NN basierte Funktion auch für den Meuse Datensatz anwenden?

In [ ]:
# create transformation

nb_pixels_x = 250
nb_pixels_y = 250

fwd_geolife = get_transform(xmax, xmin, ymax, ymin, nb_pixels_x, nb_pixels_y)

kde_array_traj_nn = np.zeros((nb_pixels_x ,nb_pixels_y))

# fit nearest neighbours



# Iterate through the ones array and consider just the values which are not no_data

show_raster_map_grid([np.log(kde_array_traj_nn+1)])
